# Introduction 

I am most interested in the work of the connectionists and neural networks. In this report, I will attempt to answer the following question:

Given an image of a Pokemon, can a convolutional neural network model be used to classify the Pokemon by type?

I had this inspiration from a dataset I recently found on Kaggle, shown here:

https://www.kaggle.com/vishalsubbiah/pokemon-images-and-types

## Background 

I refer the reader to the following Wikipedia article for more information on Pokemon:

https://en.wikipedia.org/wiki/Pok%C3%A9mon_(video_game_series)

For the purposes of context for this report, Pokemon is shorthand for Pocket Monsters, and it is a Japanese video game series developed for Nintendo gaming systems. In it, the player goes on an adventure where they assemble a team of 6 creatures, train them up to become strong, and compete for the recognition of becoming the most powerful trainer in the game world. Players in Pokemon compete by battling them against each other. As of this writing, there are 890 unique Pokemon. The Kaggle dataset mentioned above contains only 809 Pokemon, and was not updated for the additional 81 Pokemon introduced in Pokemon Sword and Shield in November 2019.

Each Pokemon has a primary and possibly a secondary type. For the purposes of complexity, we will only use the primary type of a Pokemon as a class label. There are 18 unique types of Pokemon, which each type having its own strengths and weaknesses in battle with respect to other types. Examples of types include Fire, Water, Grass, Ground, or Electric. We will investigate in this report the ability of a CNN to distinguish Pokemon by type based on their appearance in images.

# Implementation

First we load in the data:

In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy
import os

In [2]:
pokemon=pd.read_csv('pokemon.csv')
print(np.unique(pokemon['Type1'], return_counts=True))
pokemon=pokemon.sort_values('Name')
pokemon=pokemon.reset_index(drop=True)
pokemon

(array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water'], dtype=object), array([ 72,  29,  27,  40,  18,  29,  53,   3,  27,  78,  32,  23, 105,
        34,  53,  46,  26, 114], dtype=int64))


,Name,Type1,Type2
0,abomasnow,Grass,Ice
1,abra,Psychic,NaN
2,absol,Dark,NaN
3,accelgor,Bug,NaN
4,aegislash-blade,Steel,Ghost
...,...,...,...
804,zoroark,Dark,NaN
805,zorua,Dark,NaN
806,zubat,Poison,Flying
807,zweilous,Dark,Dragon


We will use the column 'Type1' for labeling. Next we load in the images:

In [ ]:
images=np.empty((len(os.listdir('images/images')), 170, 170, 3))
count=0

for root, dirs, files in os.walk('images'):
    for i, file in enumerate(files):        
        path = os.path.join(root, file) 
        img=cv2.imread(path)
        img=cv2.resize(img, (170, 170))
        images[count] = img        
        count=count+1
        print("Loaded file "+str(count)+ " of "+str(len(os.listdir('images/images')))+ " ")              

Loaded file 1 of 58154 
Loaded file 2 of 58154 
Loaded file 3 of 58154 
Loaded file 4 of 58154 
Loaded file 5 of 58154 
Loaded file 6 of 58154 
Loaded file 7 of 58154 
Loaded file 8 of 58154 
Loaded file 9 of 58154 
Loaded file 10 of 58154 
Loaded file 11 of 58154 
Loaded file 12 of 58154 
Loaded file 13 of 58154 
Loaded file 14 of 58154 
Loaded file 15 of 58154 
Loaded file 16 of 58154 
Loaded file 17 of 58154 
Loaded file 18 of 58154 
Loaded file 19 of 58154 
Loaded file 20 of 58154 
Loaded file 21 of 58154 
Loaded file 22 of 58154 
Loaded file 23 of 58154 
Loaded file 24 of 58154 
Loaded file 25 of 58154 
Loaded file 26 of 58154 
Loaded file 27 of 58154 
Loaded file 28 of 58154 
Loaded file 29 of 58154 
Loaded file 30 of 58154 
Loaded file 31 of 58154 
Loaded file 32 of 58154 
Loaded file 33 of 58154 
Loaded file 34 of 58154 
Loaded file 35 of 58154 
Loaded file 36 of 58154 
Loaded file 37 of 58154 
Loaded file 38 of 58154 
Loaded file 39 of 58154 
Loaded file 40 of 58154 
Loaded fi

Loaded file 321 of 58154 
Loaded file 322 of 58154 
Loaded file 323 of 58154 
Loaded file 324 of 58154 
Loaded file 325 of 58154 
Loaded file 326 of 58154 
Loaded file 327 of 58154 
Loaded file 328 of 58154 
Loaded file 329 of 58154 
Loaded file 330 of 58154 
Loaded file 331 of 58154 
Loaded file 332 of 58154 
Loaded file 333 of 58154 
Loaded file 334 of 58154 
Loaded file 335 of 58154 
Loaded file 336 of 58154 
Loaded file 337 of 58154 
Loaded file 338 of 58154 
Loaded file 339 of 58154 
Loaded file 340 of 58154 
Loaded file 341 of 58154 
Loaded file 342 of 58154 
Loaded file 343 of 58154 
Loaded file 344 of 58154 
Loaded file 345 of 58154 
Loaded file 346 of 58154 
Loaded file 347 of 58154 
Loaded file 348 of 58154 
Loaded file 349 of 58154 
Loaded file 350 of 58154 
Loaded file 351 of 58154 
Loaded file 352 of 58154 
Loaded file 353 of 58154 
Loaded file 354 of 58154 
Loaded file 355 of 58154 
Loaded file 356 of 58154 
Loaded file 357 of 58154 
Loaded file 358 of 58154 
Loaded file 

Loaded file 637 of 58154 
Loaded file 638 of 58154 
Loaded file 639 of 58154 
Loaded file 640 of 58154 
Loaded file 641 of 58154 
Loaded file 642 of 58154 
Loaded file 643 of 58154 
Loaded file 644 of 58154 
Loaded file 645 of 58154 
Loaded file 646 of 58154 
Loaded file 647 of 58154 
Loaded file 648 of 58154 
Loaded file 649 of 58154 
Loaded file 650 of 58154 
Loaded file 651 of 58154 
Loaded file 652 of 58154 
Loaded file 653 of 58154 
Loaded file 654 of 58154 
Loaded file 655 of 58154 
Loaded file 656 of 58154 
Loaded file 657 of 58154 
Loaded file 658 of 58154 
Loaded file 659 of 58154 
Loaded file 660 of 58154 
Loaded file 661 of 58154 
Loaded file 662 of 58154 
Loaded file 663 of 58154 
Loaded file 664 of 58154 
Loaded file 665 of 58154 
Loaded file 666 of 58154 
Loaded file 667 of 58154 
Loaded file 668 of 58154 
Loaded file 669 of 58154 
Loaded file 670 of 58154 
Loaded file 671 of 58154 
Loaded file 672 of 58154 
Loaded file 673 of 58154 
Loaded file 674 of 58154 
Loaded file 

Loaded file 953 of 58154 
Loaded file 954 of 58154 
Loaded file 955 of 58154 
Loaded file 956 of 58154 
Loaded file 957 of 58154 
Loaded file 958 of 58154 
Loaded file 959 of 58154 
Loaded file 960 of 58154 
Loaded file 961 of 58154 
Loaded file 962 of 58154 
Loaded file 963 of 58154 
Loaded file 964 of 58154 
Loaded file 965 of 58154 
Loaded file 966 of 58154 
Loaded file 967 of 58154 
Loaded file 968 of 58154 
Loaded file 969 of 58154 
Loaded file 970 of 58154 
Loaded file 971 of 58154 
Loaded file 972 of 58154 
Loaded file 973 of 58154 
Loaded file 974 of 58154 
Loaded file 975 of 58154 
Loaded file 976 of 58154 
Loaded file 977 of 58154 
Loaded file 978 of 58154 
Loaded file 979 of 58154 
Loaded file 980 of 58154 
Loaded file 981 of 58154 
Loaded file 982 of 58154 
Loaded file 983 of 58154 
Loaded file 984 of 58154 
Loaded file 985 of 58154 
Loaded file 986 of 58154 
Loaded file 987 of 58154 
Loaded file 988 of 58154 
Loaded file 989 of 58154 
Loaded file 990 of 58154 
Loaded file 

Loaded file 1259 of 58154 
Loaded file 1260 of 58154 
Loaded file 1261 of 58154 
Loaded file 1262 of 58154 
Loaded file 1263 of 58154 
Loaded file 1264 of 58154 
Loaded file 1265 of 58154 
Loaded file 1266 of 58154 
Loaded file 1267 of 58154 
Loaded file 1268 of 58154 
Loaded file 1269 of 58154 
Loaded file 1270 of 58154 
Loaded file 1271 of 58154 
Loaded file 1272 of 58154 
Loaded file 1273 of 58154 
Loaded file 1274 of 58154 
Loaded file 1275 of 58154 
Loaded file 1276 of 58154 
Loaded file 1277 of 58154 
Loaded file 1278 of 58154 
Loaded file 1279 of 58154 
Loaded file 1280 of 58154 
Loaded file 1281 of 58154 
Loaded file 1282 of 58154 
Loaded file 1283 of 58154 
Loaded file 1284 of 58154 
Loaded file 1285 of 58154 
Loaded file 1286 of 58154 
Loaded file 1287 of 58154 
Loaded file 1288 of 58154 
Loaded file 1289 of 58154 
Loaded file 1290 of 58154 
Loaded file 1291 of 58154 
Loaded file 1292 of 58154 
Loaded file 1293 of 58154 
Loaded file 1294 of 58154 
Loaded file 1295 of 58154 
L

...

Loaded file 1867 of 58154 
Loaded file 1868 of 58154 
Loaded file 1869 of 58154 
Loaded file 1870 of 58154 
Loaded file 1871 of 58154 
Loaded file 1872 of 58154 
Loaded file 1873 of 58154 
Loaded file 1874 of 58154 
Loaded file 1875 of 58154 
Loaded file 1876 of 58154 
Loaded file 1877 of 58154 
Loaded file 1878 of 58154 
Loaded file 1879 of 58154 
Loaded file 1880 of 58154 
Loaded file 1881 of 58154 
Loaded file 1882 of 58154 
Loaded file 1883 of 58154 
Loaded file 1884 of 58154 
Loaded file 1885 of 58154 
Loaded file 1886 of 58154 
Loaded file 1887 of 58154 
Loaded file 1888 of 58154 
Loaded file 1889 of 58154 
Loaded file 1890 of 58154 
Loaded file 1891 of 58154 
Loaded file 1892 of 58154 
Loaded file 1893 of 58154 
Loaded file 1894 of 58154 
Loaded file 1895 of 58154 
Loaded file 1896 of 58154 
Loaded file 1897 of 58154 
Loaded file 1898 of 58154 
Loaded file 1899 of 58154 
Loaded file 1900 of 58154 
Loaded file 1901 of 58154 
Loaded file 1902 of 58154 
Loaded file 1903 of 58154 
L

Loaded file 2171 of 58154 
Loaded file 2172 of 58154 
Loaded file 2173 of 58154 
Loaded file 2174 of 58154 
Loaded file 2175 of 58154 
Loaded file 2176 of 58154 
Loaded file 2177 of 58154 
Loaded file 2178 of 58154 
Loaded file 2179 of 58154 
Loaded file 2180 of 58154 
Loaded file 2181 of 58154 
Loaded file 2182 of 58154 
Loaded file 2183 of 58154 
Loaded file 2184 of 58154 
Loaded file 2185 of 58154 
Loaded file 2186 of 58154 
Loaded file 2187 of 58154 
Loaded file 2188 of 58154 
Loaded file 2189 of 58154 
Loaded file 2190 of 58154 
Loaded file 2191 of 58154 
Loaded file 2192 of 58154 
Loaded file 2193 of 58154 
Loaded file 2194 of 58154 
Loaded file 2195 of 58154 
Loaded file 2196 of 58154 
Loaded file 2197 of 58154 
Loaded file 2198 of 58154 
Loaded file 2199 of 58154 
Loaded file 2200 of 58154 
Loaded file 2201 of 58154 
Loaded file 2202 of 58154 
Loaded file 2203 of 58154 
Loaded file 2204 of 58154 
Loaded file 2205 of 58154 
Loaded file 2206 of 58154 
Loaded file 2207 of 58154 
L

Loaded file 2475 of 58154 
Loaded file 2476 of 58154 
Loaded file 2477 of 58154 
Loaded file 2478 of 58154 
Loaded file 2479 of 58154 
Loaded file 2480 of 58154 
Loaded file 2481 of 58154 
Loaded file 2482 of 58154 
Loaded file 2483 of 58154 
Loaded file 2484 of 58154 
Loaded file 2485 of 58154 
Loaded file 2486 of 58154 
Loaded file 2487 of 58154 
Loaded file 2488 of 58154 
Loaded file 2489 of 58154 
Loaded file 2490 of 58154 
Loaded file 2491 of 58154 
Loaded file 2492 of 58154 
Loaded file 2493 of 58154 
Loaded file 2494 of 58154 
Loaded file 2495 of 58154 
Loaded file 2496 of 58154 
Loaded file 2497 of 58154 
Loaded file 2498 of 58154 
Loaded file 2499 of 58154 
Loaded file 2500 of 58154 
Loaded file 2501 of 58154 
Loaded file 2502 of 58154 
Loaded file 2503 of 58154 
Loaded file 2504 of 58154 
Loaded file 2505 of 58154 
Loaded file 2506 of 58154 
Loaded file 2507 of 58154 
Loaded file 2508 of 58154 
Loaded file 2509 of 58154 
Loaded file 2510 of 58154 
Loaded file 2511 of 58154 
L

Loaded file 2779 of 58154 
Loaded file 2780 of 58154 
Loaded file 2781 of 58154 
Loaded file 2782 of 58154 
Loaded file 2783 of 58154 
Loaded file 2784 of 58154 
Loaded file 2785 of 58154 
Loaded file 2786 of 58154 
Loaded file 2787 of 58154 
Loaded file 2788 of 58154 
Loaded file 2789 of 58154 
Loaded file 2790 of 58154 
Loaded file 2791 of 58154 
Loaded file 2792 of 58154 
Loaded file 2793 of 58154 
Loaded file 2794 of 58154 
Loaded file 2795 of 58154 
Loaded file 2796 of 58154 
Loaded file 2797 of 58154 
Loaded file 2798 of 58154 
Loaded file 2799 of 58154 
Loaded file 2800 of 58154 
Loaded file 2801 of 58154 
Loaded file 2802 of 58154 
Loaded file 2803 of 58154 
Loaded file 2804 of 58154 
Loaded file 2805 of 58154 
Loaded file 2806 of 58154 
Loaded file 2807 of 58154 
Loaded file 2808 of 58154 
Loaded file 2809 of 58154 
Loaded file 2810 of 58154 
Loaded file 2811 of 58154 
Loaded file 2812 of 58154 
Loaded file 2813 of 58154 
Loaded file 2814 of 58154 
Loaded file 2815 of 58154 
L

Loaded file 3083 of 58154 
Loaded file 3084 of 58154 
Loaded file 3085 of 58154 
Loaded file 3086 of 58154 
Loaded file 3087 of 58154 
Loaded file 3088 of 58154 
Loaded file 3089 of 58154 
Loaded file 3090 of 58154 
Loaded file 3091 of 58154 
Loaded file 3092 of 58154 
Loaded file 3093 of 58154 
Loaded file 3094 of 58154 
Loaded file 3095 of 58154 
Loaded file 3096 of 58154 
Loaded file 3097 of 58154 
Loaded file 3098 of 58154 
Loaded file 3099 of 58154 
Loaded file 3100 of 58154 
Loaded file 3101 of 58154 
Loaded file 3102 of 58154 
Loaded file 3103 of 58154 
Loaded file 3104 of 58154 
Loaded file 3105 of 58154 
Loaded file 3106 of 58154 
Loaded file 3107 of 58154 
Loaded file 3108 of 58154 
Loaded file 3109 of 58154 
Loaded file 3110 of 58154 
Loaded file 3111 of 58154 
Loaded file 3112 of 58154 
Loaded file 3113 of 58154 
Loaded file 3114 of 58154 
Loaded file 3115 of 58154 
Loaded file 3116 of 58154 
Loaded file 3117 of 58154 
Loaded file 3118 of 58154 
Loaded file 3119 of 58154 
L

In [ ]:
images.shape

Some Pokemon types are intuitive and some are not. For example, the first and third Pokemon in the original dataset are Ice-type Pokemon, which is supported by their white, snowy appearance. However, the second Pokemon is Psychic-type, which is not immediately evident from its appearance. The difficulty of this task for humans is present because of this fact. Thus, we hope to see how difficult this task is for a neural network.

Note that the original dataset contains only 809 images in 18 classes. This is hardly enough data with which to train a model. Thus, I artificially generate new images by rotating the given ones by 5 degrees, saving them, and adding them to our dataset.

Next we preprocess the data: